<a href="https://colab.research.google.com/github/narsym/deep-learning-with-tensorflow-2.0/blob/master/Converting_model_to_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Converting model to estimator

In [0]:
import os
import numpy as np
import time
import tensorflow as tf

**Getting Fashion Mnist data**

In [2]:
LABEL_DIMENSIONS = 10
(train_images,train_labels), (test_images,test_labels) = tf.keras.datasets.fashion_mnist.load_data()
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

4423680/4422102 [==============================] - 0s 0us/step


Type casting training and testing images into float32

In [0]:
train_images = np.asarray(train_images,dtype = np.float32) / 255
test_images = np.asarray(test_images,dtype = np.float32) / 255

Reshaping Train and Test Images

In [0]:
train_images = train_images.reshape((TRAINING_SIZE,28,28,1))
test_images = test_images.reshape((TEST_SIZE,28,28,1))

Creating one hot encodings for train and test labels

In [0]:
train_labels = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

Type casting train and test labels

In [7]:
train_labels = np.asarray(train_labels,dtype = np.float32)
test_labels = np.asarray(test_labels,dtype = np.float32)
print(train_labels.shape,test_labels.shape)  

(60000, 10) (10000, 10)


#Conv Model with Functional API

In [13]:
inputs = tf.keras.Input(shape = (28,28,1))
x = tf.keras.layers.Conv2D(filters = 32,kernel_size = (3,3),activation = 'relu')(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides = 2)(x)
x = tf.keras.layers.Conv2D(filters = 64,kernel_size = (3,3),activation = 'relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides = 2)(x)
x = tf.keras.layers.Conv2D(filters = 64,kernel_size = (3,3),activation = 'relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64,activation = 'relu')(x)
prediction = tf.keras.layers.Dense(LABEL_DIMENSIONS,activation = 'softmax')(x)
model = tf.keras.models.Model(inputs = inputs,outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0     

Optimizer and compiling

In [0]:
optimizer = tf.keras.optimizers.SGD()
model.compile(optimizer = optimizer,loss = 'categorical_crossentropy',metrics = ['accuracy'])

**STRATEGY for ESTIMATOR (either cpu or gpu)**

In [20]:
strategy = None #CPU
#strategy = tf.distribute.MirroredStrategy() # GPU
config = tf.estimator.RunConfig(train_distribute=strategy)
estimator = tf.keras.estimator.model_to_estimator(model,config = config)

INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7zguglbn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Creating dataset for estimator

In [0]:
def input_fn(images,labels,epochs,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((images,labels))
  SHUFFLE_SIZE = 5000
  dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
  dataset = dataset.prefetch(None)
  return dataset

Training estimator on Cpu

In [22]:
BATCH_SIZE = 512
EPOCHS = 50
estimator_train_result = estimator.train(input_fn = lambda:input_fn(train_images,train_labels,epochs = EPOCHS,batch_size = BATCH_SIZE))
print(estimator_train_result)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmp7zguglbn/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmp7zguglbn/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7zguglbn/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after sa

Estimator result cpu

In [24]:
estimator.evaluate(lambda:input_fn(test_images,test_labels,EPOCHS,BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-31T14:10:08Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7zguglbn/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 99.89179s
INFO:tensorflow:Finished evaluation at 2020-05-31-14:11:48
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.843, global_step = 5860, loss = 0.4400091
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmp7zguglbn/model.ckpt-5860


{'accuracy': 0.843, 'global_step': 5860, 'loss': 0.4400091}